# Search for and download STAC items

Retrieve STAC items from your storage using different search filters and download image assets.

You can make a [detailed search request](https://docs.up42.com/developers/api-stac#detailed-search) to find specific STAC items in your storage.

## Set up the notebook

### 1. Install dependencies

In [ ]:
!pip install up42-py --upgrade -q

import up42, pathlib

### 2. Configure credentials

Run the cell below to create a `credentials.json` file in a directory named `.up42` in your home folder.

In [ ]:
# Define the credentials file path if it doesn't exist
credentials_file_path = pathlib.Path.home().joinpath(".up42/credentials.json")
credentials_file_path.parent.mkdir(parents=True, exist_ok=True)
credentials_file_path.touch(exist_ok=True)

1. Click the link above to the created file and paste the following code:
    ```json
    {
        "username": "<your-email-address>",
        "password": "<your-password>"
    }
    ```
1. Retrieve the email address and password used for logging into the console. Use them as values for `username` and `password`.
1. Save the `credentials.json` file.

### 3. Authenticate and create a PySTAC client connection

In [ ]:
up42.authenticate(cfg_file=credentials_file_path)

# Use region if you're logging with the NSG service
# up42.authenticate(cfg_file=credentials_file_path, region="sa")

UP42_client = up42.stac_client()

## Search for STAC items

### Option 1. Search by STAC item ID

In [ ]:
# Define a STAC item ID
item_id = "68567134-27ad-7bd7-4b65-d61adb11fc78"

# Perform the search
item = next(UP42_client.get_items(item_id))

# Print the result
print(f"STAC item ID:   {item.id}")
print(f"Order ID:       {item.up42.order_id}")
print(f"User title:     {item.up42.title}")
print(f"User tags:      {item.up42.tags}\n")

### Option 2. Search by order ID

In [ ]:
# Define an order ID
order_id = "68567134-27ad-7bd7-4b65-d61adb11fc78"

# This filter searches for STAC items associated with the defined order ID
filter = {
    "op": "=",
    "args": [
        {"property": "up42-order:id"},
        order_id,
    ],
}

# Perform the search
stac_items_search = UP42_client.search(filter=filter)

# Iterate through each STAC item and print selected details
for item in stac_items_search.items():
    print(f"STAC item ID:   {item.id}")
    print(f"Order ID:       {item.up42.order_id}")
    print(f"User title:     {item.up42.title}")
    print(f"User tags:      {item.up42.tags}\n")

### Option 3. Search by tags

In [ ]:
# Set up a filter to search for STAC items that contain the "ortho" tag
filter = {
    "op": "a_contains",
    "args": [
        {"property": "tags"},
        ["ortho"],
    ],
}

# Perform the search
stac_items_search = UP42_client.search(filter=filter)

for item in stac_items_search.items():
    properties = item.properties

    # Print all results matching the search filter
    print(f"STAC item ID:   {item.id}")
    print(f"Order ID:       {item.up42.order_id}")
    print(f"User title:     {item.up42.title}")
    print(f"User tags:      {item.up42.tags}\n")

### Option 4. Search for STAC items from a specific data product

In [ ]:
# Retrieve a list of geospatial collections
data_collections = up42.ProductGlossary.get_collections(
    collection_type=up42.CollectionType.ARCHIVE,
)

# Iterate over collections and select SPOT
collection = next(c for c in data_collections if c.name == "spot")

# Find the Display data product
data_product = next(d for d in collection.data_products if d.name == "spot-display")

# Set up a filter to search for spot-display items in storage
filter = {"op": "=", "args": [{"property": "up42-product:product_id"}, data_product.id]}

# Perform the search and iterate to next item
stac_items_search = UP42_client.search(filter=filter)
for item in stac_items_search.items():
    properties = item.properties

    # Print all results matching the search filter
    print(f"STAC item ID:   {item.id}")
    print(f"Order ID:       {item.up42.order_id}")
    print(f"User title:     {item.up42.title}")
    print(f"User tags:      {item.up42.tags}\n")

### Option 5. Search for all STAC items in storage

You can retrieve all STAC items from your storage. Don't use if you have more than 100 STAC items in storage.

In [ ]:
# Fetch all STAC items
items = UP42_client.get_items()

# Print results
item = next(items)
print(f"STAC item ID:   {item.id}")
print(f"Order ID:       {item.up42.order_id}")
print(f"User title:     {item.up42.title}")
print(f"User tags:      {item.up42.tags}\n")

## Download the image asset

STAC items usually contain several assets. Let's select and dowload the RGB multispectral asset.

In [ ]:
# Select a STAC item
item_id = "2e20dd4b-7a2c-4197-96e6-69fd8859565c"
item = next(UP42_client.get_items(item_id))

# Find the RGB image asset with desired bands
rgb_bands = {"red", "green", "blue"}
rgb_key = None
for key, asset in item.assets.items():
    bands = asset.extra_fields.get("eo:bands")
    if not bands:
        continue
    band_names = set([band["name"] for band in bands])
    if rgb_bands.issubset(band_names):
        rgb_key = key
        break

rgb_asset = item.assets[rgb_key]

# Download the RGB image asset to your home folder
rgb_asset.file.download(output_directory=pathlib.Path.home() / "Desktop")